# Get Books
* Return books for authors using different APIs

In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import requests
import yaml

In [2]:
google_key_file = "/Users/alicia.brown/google_books.yaml"
with open(google_key_file, 'r') as stream:
    google_config = yaml.load(stream)
    google_key = google_config['key']

In [6]:
authors_file = "../data/books - authors.csv"
authors = pd.read_csv(authors_file)

## Google Books
* https://developers.google.com/books/docs/overview
* Example: https://www.googleapis.com/books/v1/volumes?q=inauthor:louise+penny&langRestrict=en&orderBy=newest&maxresults=40

![Google Books](https://books.google.com/googlebooks/images/poweredby.png "Google Books")

In [4]:
def getGoogleBooks(author_first, author_last, order_by):
    url = "https://www.googleapis.com/books/v1/volumes?q=inauthor:" + author_first + "+" + author_last + "&langRestrict=en&orderBy=" + order_by + "&printType=books&maxresults=40&prettyPrint=true&key=" + google_key
    req = requests.get(url)
    results = json.loads(req.text)
    df = json_normalize(results['items'])
    return df

### Iterate through Authors

In [5]:
author_results = []
author_list = []
for row in authors.itertuples():
    df_results = getGoogleBooks(row[1], row[2], "relevance")
    author_results.append(df_results)
    # keeping track of Author names to compare results against
    author_list.append(row[1] + ' ' + row[2])
df = pd.concat(author_results)
df.tail()

,accessInfo.accessViewStatus,accessInfo.country,accessInfo.embeddable,accessInfo.epub.acsTokenLink,accessInfo.epub.isAvailable,accessInfo.pdf.acsTokenLink,accessInfo.pdf.isAvailable,accessInfo.publicDomain,accessInfo.quoteSharingAllowed,accessInfo.textToSpeechPermission,...,volumeInfo.panelizationSummary.containsImageBubbles,volumeInfo.previewLink,volumeInfo.printType,volumeInfo.publishedDate,volumeInfo.publisher,volumeInfo.ratingsCount,volumeInfo.readingModes.image,volumeInfo.readingModes.text,volumeInfo.subtitle,volumeInfo.title
5,NONE,US,False,NaN,False,NaN,False,False,False,ALLOWED,...,NaN,http://books.google.com/books?id=sP5vRAAACAAJ&...,BOOK,1969,NaN,NaN,False,False,NaN,The Christmas Tree
6,SAMPLE,US,True,http://books.google.com/books/download/Messeng...,True,NaN,False,False,False,ALLOWED,...,NaN,http://books.google.com/books?id=8aNr_Y4KIvcC&...,BOOK,2010-04-01,Henry Holt and Company,52.0,False,True,A Maisie Dobbs Novel,Messenger of Truth
7,SAMPLE,US,True,http://books.google.com/books/download/Pardona...,True,NaN,False,False,False,ALLOWED,...,NaN,http://books.google.com/books?id=2771InPRaQ4C&...,BOOK,2007-04-01,Henry Holt and Company,54.0,False,True,A Maisie Dobbs Novel,Pardonable Lies
8,SAMPLE,US,True,http://books.google.com/books/download/Elegy_f...,True,NaN,False,False,False,ALLOWED_FOR_ACCESSIBILITY,...,False,http://books.google.com/books?id=MfArUIGJQDYC&...,BOOK,2012-03-27,Harper Collins,85.0,False,True,A Maisie Dobbs Novel,Elegy for Eddie
9,SAMPLE,US,True,http://books.google.com/books/download/A_Lesso...,True,NaN,False,False,False,ALLOWED_FOR_ACCESSIBILITY,...,False,http://books.google.com/books?id=BlJvZL__KzcC&...,BOOK,2011-03-22,Harper Collins,12.0,False,True,A Maisie Dobbs Novel,A Lesson in Secrets


### Finalize Results
* Select fields for analysis
* Drop `volumeInfo.` and `searchInfo.` from field names
* Convert `authors` to string

In [227]:
df_final = df[['volumeInfo.authors','volumeInfo.title','volumeInfo.subtitle','volumeInfo.publishedDate','volumeInfo.publisher','volumeInfo.contentVersion','volumeInfo.averageRating','volumeInfo.ratingsCount','volumeInfo.description','searchInfo.textSnippet']].sort_values(by=['volumeInfo.publishedDate'], ascending = False)
df_final.rename(columns=lambda x: x.replace('volumeInfo.',''), inplace=True)
df_final.rename(columns=lambda x: x.replace('searchInfo.',''), inplace=True)
df_final['authors'] = df_final['authors'].apply(lambda x: ','.join(map(str, x)))

In [229]:
df_final.describe(include = 'all')

,authors,title,subtitle,publishedDate,publisher,contentVersion,averageRating,ratingsCount,description,textSnippet
count,50,50,23,50,49,50,43.000000,43.000000,48,49
unique,6,50,5,45,13,31,NaN,NaN,48,49
top,Fred Vargas,The Purity of Vengeance,A Chief Inspector Gamache Novel,2010-04-01,Penguin,preview-1.0.0,NaN,NaN,Penguin's popular Aunt Dimity series returns a...,Maisie Dobbs takes on her first investigation ...
freq,10,1,7,3,18,11,NaN,NaN,1,1
mean,NaN,NaN,NaN,NaN,NaN,NaN,3.825581,140.395349,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,0.555103,514.639871,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,2.500000,1.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,3.500000,5.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,14.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,47.000000,NaN,NaN


In [245]:
# Check expected Author count vs Search Results
print(len(authors) == len(df_final['authors'].unique()))

False


In [231]:
df_final.to_csv("../data/book_search_results.csv", index=False, na_rep='', quotechar='"', encoding='utf-8')

## Check for New Books

In [7]:
author_results = []
author_list = []
for row in authors.itertuples():
    df_results = getGoogleBooks(row[1], row[2], "newest")
    author_results.append(df_results)
    # keeping track of Author names to compare results against
    author_list.append(row[1] + ' ' + row[2])
df = pd.concat(author_results)
df.tail()

,accessInfo.accessViewStatus,accessInfo.country,accessInfo.embeddable,accessInfo.epub.acsTokenLink,accessInfo.epub.isAvailable,accessInfo.pdf.acsTokenLink,accessInfo.pdf.isAvailable,accessInfo.publicDomain,accessInfo.quoteSharingAllowed,accessInfo.textToSpeechPermission,...,volumeInfo.publishedDate,volumeInfo.publisher,volumeInfo.ratingsCount,volumeInfo.readingModes.image,volumeInfo.readingModes.text,volumeInfo.seriesInfo.bookDisplayNumber,volumeInfo.seriesInfo.kind,volumeInfo.seriesInfo.volumeSeries,volumeInfo.subtitle,volumeInfo.title
5,NONE,US,False,NaN,False,NaN,False,False,False,ALLOWED,...,2017-11,NaN,NaN,False,False,NaN,NaN,NaN,NaN,LP'S BEST OF AUSTRALIA 2
6,NONE,US,False,NaN,False,NaN,False,False,False,ALLOWED,...,2015,NaN,NaN,False,False,NaN,NaN,NaN,A Novel,A God in Ruins
7,NONE,US,False,NaN,False,NaN,False,False,False,ALLOWED,...,2017-07-12,NaN,NaN,False,False,NaN,NaN,NaN,NaN,"Thinking in the Heart, Or Easy Lessons in Real..."
8,NONE,US,False,NaN,False,NaN,False,False,False,ALLOWED,...,2017-05-18,Forgotten Books,NaN,False,False,NaN,NaN,NaN,NaN,New Thought Healing Made Plain (Classic Reprint)
9,SAMPLE,US,False,http://books.google.com/books/download/Life_Af...,True,NaN,False,False,False,ALLOWED,...,2013-04-02,Reagan Arthur Books,272.0,False,True,NaN,NaN,NaN,A Novel,Life After Life


In [8]:
df[,]

,accessInfo.accessViewStatus,accessInfo.country,accessInfo.embeddable,accessInfo.epub.acsTokenLink,accessInfo.epub.isAvailable,accessInfo.pdf.acsTokenLink,accessInfo.pdf.isAvailable,accessInfo.publicDomain,accessInfo.quoteSharingAllowed,accessInfo.textToSpeechPermission,...,volumeInfo.publishedDate,volumeInfo.publisher,volumeInfo.ratingsCount,volumeInfo.readingModes.image,volumeInfo.readingModes.text,volumeInfo.seriesInfo.bookDisplayNumber,volumeInfo.seriesInfo.kind,volumeInfo.seriesInfo.volumeSeries,volumeInfo.subtitle,volumeInfo.title
0,NONE,US,False,NaN,False,NaN,False,False,False,ALLOWED,...,2018-11-27,Minotaur Books,NaN,False,False,NaN,NaN,NaN,A Chief Inspector Gamache Novel,Kingdom of the Blind
1,NONE,US,False,NaN,False,NaN,False,False,False,ALLOWED,...,2018-11-27,Minotaur Books,NaN,False,True,NaN,NaN,NaN,A Chief Inspector Gamache Novel,Kingdom of the Blind
2,SAMPLE,US,True,http://books.google.com/books/download/CACHE_L...,True,NaN,False,False,False,ALLOWED,...,2017-11-17,Hodder Education,NaN,False,True,NaN,NaN,NaN,NaN,CACHE Level 2 Award in Child Development and Care
3,SAMPLE,US,True,http://books.google.com/books/download/Glass_H...,True,NaN,False,False,False,ALLOWED,...,2017-08-29,Minotaur Books,40.0,False,True,NaN,NaN,NaN,A Novel,Glass Houses
4,NONE,US,False,NaN,False,NaN,False,False,False,ALLOWED,...,2018-05-01,Minotaur Books,40.0,False,False,NaN,NaN,NaN,A Novel,Glass Houses
5,NONE,US,False,NaN,False,NaN,False,False,False,ALLOWED,...,2017-08-29,Sphere,40.0,False,False,NaN,NaN,NaN,NaN,Glass Houses
6,SAMPLE,US,True,NaN,False,NaN,False,False,False,ALLOWED,...,2017-08-29,Minotaur Books,40.0,False,False,NaN,NaN,NaN,A Novel,Glass Houses
7,NONE,US,False,NaN,False,NaN,False,False,False,ALLOWED,...,2018-11-27,Minotaur,NaN,False,False,NaN,NaN,NaN,A Chief Inspector Gamache Novel,Kingdom of the Blind
8,NONE,US,False,NaN,False,NaN,False,False,False,ALLOWED,...,2018-11-27,Sphere,NaN,False,False,NaN,NaN,NaN,A Chief Inspector Gamache Mystery,Untitled Louise Penny 2
9,NONE,US,False,NaN,True,NaN,True,False,False,ALLOWED,...,2017-11-06,Hachette UK,NaN,False,True,NaN,NaN,NaN,NaN,CACHE Level 2 Award in Child Development and Care
